In [ ]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(palette='pastel')
plt.rcParams['figure.figsize'] = (12, 6)
pd.set_option('display.max_columns', None)

In [ ]:
df = pd.read_csv("../data/processed/procedimientos_procesada.csv")

In [ ]:
df["fecha"] = pd.to_datetime(df["fecha"], errors="coerce")
df = df.dropna(subset="fecha")

In [ ]:
produccion_radiografia = (
    df.query("accion_clinica == 'RADIOGRAFIA DE TORAX FRONTAL Y LATERAL'")
    .groupby("fecha")["numero_de_veces"]
    .sum()
)

In [ ]:
produccion_radiografia_mensual = produccion_radiografia.resample("M").sum()